In [95]:
import requests
import pandas as pd
import json
import psycopg2
from typing import Any

In [101]:
restcountries_url = "https://restcountries.com/v3.1/all"

def get_data(url:str) -> json:
  request = requests.get(url)
  if request.status_code == 200:
    response = request.json()
    return response
  else:
      return('Error fecthing data')


def transform_data(data: Any) -> pd.DataFrame:
    # Extract relevant fields from each dictionary in the list
    countries = {
        'country_name': [info['name']['common'] for info in data],
        'independence': [info.get('independent') for info in data],
        'un_members': [info.get('unMember') for info in data],
        'startOfWeek': [info.get('startOfWeek') for info in data],
        'official_country_name': [info['name'].get('official') for info in data],
        'common_native_names': [info['name'].get('nativeName', {}).get('eng', {}).get('common') for info in data],

        # Handle currency details
        'currency_code': [list(info.get('currencies', {}).keys())[0] if info.get('currencies') else None for info in data],
        'currency_name': [list(info.get('currencies', {}).values())[0].get('name') if info.get('currencies') else None for info in data],
        'currency_symbol': [list(info.get('currencies', {}).values())[0].get('symbol') if info.get('currencies') else None for info in data],

        'country_code': [info.get('cca3') for info in data],
        'capital': [info.get('capital', [None])[0] for info in data],
        'region': [info.get('region') for info in data],
        'sub_region': [info.get('subregion') for info in data],

        # Join languages into a single string
        'languages': [', '.join(info.get('languages', {}).values()) for info in data],

        'area': [info.get('area') for info in data],
        'population': [info.get('population') for info in data],

        # Extract first continent
        'continents': [info.get('continents', [None])[0] for info in data],

        # Handle latlng safely
        'longitude': [info.get('latlng', [None, None])[1] for info in data],
        'latitude': [info.get('latlng', [None, None])[0] for info in data]
    }

    # Convert dictionary to DataFrame
    df_data = pd.DataFrame(countries)

    return df_data



def create_db_table(cur):
    cur.execute("""
        DROP TABLE IF EXISTS world_countries;

        CREATE TABLE IF NOT EXISTS world_countries (
            country_code VARCHAR(255) PRIMARY KEY,
            country_name VARCHAR(255),
            independence BOOLEAN,
            un_members BOOLEAN,
            start_of_week TEXT,
            official_country_name VARCHAR(255),
            common_native_name VARCHAR(255),
            currency_code VARCHAR(10),
            currency_name VARCHAR(255),
            currency_symbol VARCHAR(10),
            capital VARCHAR(255),
            region VARCHAR(255),
            sub_region VARCHAR(255),
            languages TEXT,
            area FLOAT,
            population INT,
            continents VARCHAR(255)
        )
    """)



def insert_data_to_db(df:pd.DataFrame, cur,conn):
    try:
        # Iterate over DataFrame rows as (index, Series) pairs
        for _, row in df.iterrows():
            cur.execute("""
                INSERT INTO world_countries (
                    country_code,
                    country_name,
                    independence,
                    un_members,
                    start_of_week,
                    official_country_name,
                    common_native_name,
                    currency_code,
                    currency_name,
                    currency_symbol,
                    capital,
                    region,
                    sub_region,
                    languages,
                    area,
                    population,
                    continents
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                row['country_code'],
                row['country_name'],
                row['independence'],
                row['un_members'],
                row['start_of_week'],
                row['official_country_name'],
                row['common_native_name'],
                row['currency_code'],
                row['currency_name'],
                row['currency_symbol'],
                row['capital'],
                row['region'],
                row['sub_region'],
                row['languages'],
                row['area'],
                row['population'],
                row['continents']
            ))

        # Commit the transaction
        cur.conn.commit()

    except Exception as e:
        print(f"An error occurred: {e}")



def main():
  conn = psycopg2.connect(
            "host=localhost dbname=world_countries user=postgres password=postgres")
  cur = conn.cursor()
  data = get_data(url=restcountries_url)
  transformed_data = trasnfrom_data(data)
  create_db_table(conn,cur)
  insert_data_to_db(conn,cur)
  pass


if __name__ == '__main__':
  main()

In [102]:
data = get_data(url=restcountries_url)
transformed_data =  transform_data(data)
transformed_data.iloc[0]

country_name                                            South Georgia
independence                                                    False
un_members                                                      False
startOfWeek                                                    monday
official_country_name    South Georgia and the South Sandwich Islands
common_native_names                                     South Georgia
currency_code                                                     SHP
currency_name                                      Saint Helena pound
currency_symbol                                                     £
country_code                                                      SGS
capital                                             King Edward Point
region                                                      Antarctic
sub_region                                                       None
languages                                                     English
area                

In [92]:
data[0]

{'name': {'common': 'South Georgia',
  'official': 'South Georgia and the South Sandwich Islands',
  'nativeName': {'eng': {'official': 'South Georgia and the South Sandwich Islands',
    'common': 'South Georgia'}}},
 'tld': ['.gs'],
 'cca2': 'GS',
 'ccn3': '239',
 'cca3': 'SGS',
 'independent': False,
 'status': 'officially-assigned',
 'unMember': False,
 'currencies': {'SHP': {'name': 'Saint Helena pound', 'symbol': '£'}},
 'idd': {'root': '+5', 'suffixes': ['00']},
 'capital': ['King Edward Point'],
 'altSpellings': ['GS', 'South Georgia and the South Sandwich Islands'],
 'region': 'Antarctic',
 'languages': {'eng': 'English'},
 'translations': {'ara': {'official': 'جورجيا الجنوبية وجزر ساندوتش الجنوبية',
   'common': 'جورجيا الجنوبية'},
  'bre': {'official': 'Georgia ar Su hag Inizi Sandwich ar Su',
   'common': 'Georgia ar Su hag Inizi Sandwich ar Su'},
  'ces': {'official': 'Jižní Georgie a Jižní Sandwichovy ostrovy',
   'common': 'Jižní Georgie a Jižní Sandwichovy ostrovy'},
  